In [ ]:
import twint
import pandas as pd

# sentiment analysis
from optimus import Optimus
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

op = Optimus()

# For notebooks 
import nest_asyncio
nest_asyncio.apply()

https://towardsdatascience.com/analyzing-tweets-with-nlp-in-minutes-with-spark-optimus-and-twint-a0c96084995f

'Twitter has a limitation on the number of characters used per tweet, resulting in data sparsity.' 
https://www.sciencedirect.com/science/article/pii/S1319157821002184

In [ ]:
#configuration
config = twint.Config()
config.Search = "bitcoin"
#config.Lang = "en"
config.Limit = 1000
config.Since = "2016-04-29 00:00:00" #datetime.strptime("2019-04-29 00:00:00", '%Y-%m-%d %H:%M:%S')
config.Until = "2020-04-29 12:00:00" # datetime.strptime("2020–04–29 12:00:00", '%Y-%m-%d %H:%M:%S')
config.Min_likes = 1000
config.Pandas = True
config.Hide_output = True

#running search
twint.run.Search(config)

## SEARCH ONLY RETURNED 20 RESULTS MAXIMUM, DID SOME SEARCHING AND FOUND OUT
# Uncommenting line 92 in the url.py file: ('query_source', 'typed_query'), seems to increase the amount of results


In [ ]:
#https://github.com/FavioVazquez/twitter_optimus_twint/blob/master/AnalyzeTweets.ipynb

# return all the columns in the twint output
def available_columns():
    return twint.output.panda.Tweets_df.columns

# return only the needed columns for analysis
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

# Function to get sentiment
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0  # Neutral
    elif temp >= 0.0:
        return 1.0  # Positive
    else:
        return 2.0  # Negative

# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())

def create_optimus(df):
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf=df)

    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet").cols.remove_special_chars("tweet")

    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment", sentiment(clean_tweets['tweet']))

In [ ]:
try:
    print(available_columns())
    df = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    print(df.head())
    print(df.count())
except KeyError as e:
    print("No data retrieved!")

In [ ]:
# create the Optimus DataFrame 
op_df = create_optimus(df)
op_df.printSchema()

In [ ]:
# ERROR: Java error,
# IllegalArgumentException: 'Unsupported class file major version 57'
op_df.show()